# **1.7Going Deeper On Molecular Featurizations**

In [1]:
!pip install -qq --pre deepchem

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.


In [11]:
import warnings
import os

# 1. 파이썬 기본 경고 숨기기
warnings.filterwarnings('ignore')

In [13]:
import deepchem as dc

featurizer = dc.feat.CircularFingerprint()
print(featurizer(['CC', 'CCC', 'CCO']))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


[05:00:50] DEPRECATION WARNING: please use MorganGenerator
[05:00:50] DEPRECATION WARNING: please use MorganGenerator
[05:00:50] DEPRECATION WARNING: please use MorganGenerator


직접 분자리스트를 집어넣을 있음,

SMILES는 2차원적인 정보만 주기 때문에, ConformerGenerator을 이용해 3차원 좌표정보를 줌

In [3]:
from rdkit.Chem.Descriptors import descList
rdkit_featurizer = dc.feat.RDKitDescriptors()
features = rdkit_featurizer(['CCC'])[0]
descriptors = [i[0] for i in descList]
for feature, descriptor in zip(features[:10], descriptors):
    print(descriptor, feature)

MaxAbsEStateIndex 0.9182958340544894
MaxEStateIndex 1.3523744386698802
MinAbsEStateIndex -1.4760582438091696
MinEStateIndex 1.55881365076144
qed -1.2696136507614397
SPS 3.9177136507614425
MolWt 1.089286349238561
HeavyAtomMolWt 13.425713650761436
ExactMolWt 10.597286349238557
NumValenceElectrons 1.6329931618554523


In [4]:
print('The number of descriptors present is: ', len(features))

The number of descriptors present is:  217


RDKitDescriptors를 이용해 분자의 정보 추출

GraphConv이외에도 입력양식에 따라 Weave/MolGraphConv등 다양한 모델 선택

In [5]:
from rdkit import Chem

generator = dc.utils.ConformerGenerator(max_conformers=5)
propane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCC'))
print("Number of available conformers for propane: ", len(propane_mol.GetConformers()))

Number of available conformers for propane:  1


In [6]:
butane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCCC'))
print("Number of available conformers for butane: ", len(butane_mol.GetConformers()))

Number of available conformers for butane:  3


쿨롱행렬을 만들기 위해 smiles를 이용해 3D입체구조 생성
-> 원자들 사이의 거리 측정

CCCC는 3개의 다른모양 형성

In [7]:
coulomb_mat = dc.feat.CoulombMatrix(max_atoms=20)
features = coulomb_mat(propane_mol)
print(features)

[[[36.8581052  12.48684436  7.56196875  2.85804542  2.85945164
    2.85804541  1.46740143  1.46740146  0.91279488  1.14239695
    1.14239686  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [12.48684436 36.8581052  12.48684457  1.45850729  1.46551208
    1.45850733  2.856895    2.85689499  1.46551214  1.45850729
    1.45850735  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 7.56196875 12.48684457 36.8581052   1.14239716  0.91279487
    1.14239667  1.46740155  1.46740152  2.8594516   2.85804528
    2.85804531  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 2.85804542  1.45850729  1.14239716  0.5         0.29325368
    0.29200274  0.17113412  0.21092508  0.13960187  0.16800025
    0.20540035  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 2.85945164  1.46

/usr/local/lib/python3.12/dist-packages/deepchem/feat/molecule_featurizers/coulomb_matrices.py:151: RuntimeWarning: divide by zero encountered in divide
  m = np.outer(z, z) / d


In [8]:
coulomb_mat_eig = dc.feat.CoulombMatrixEig(max_atoms=20)
features = coulomb_mat_eig(propane_mol)
print(features)

[[60.07620356 29.62963145 22.75497727  0.57137861  0.28781334  0.28548339
   0.27558185  0.18163797  0.17460999  0.17059716  0.16640101  0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]]


Max atoms를 정해주는 이유는 입력 크기를 일정하게 Padding해주기 위해

CoulombMatrix는 2차원 CoulombMatrixEIg는 1차원 출력

1차원은 순서가 바뀌어도 데이터가 변하지 않는 안정성 지님.(PCA와 비슷)

2차원은 정밀한 예측 but 데이터가 크고 순서에 민감

#**Smiles**

In [9]:
import numpy as np

In [12]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer="Raw")
train_dataset, valid_dataset, test_dataset = datasets
print(train_dataset)

<DiskDataset X.shape: (6258,), y.shape: (6258, 12), w.shape: (6258, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>


In [14]:
train_smiles = train_dataset.ids
valid_smiles = valid_dataset.ids
test_smiles = test_dataset.ids
print(train_smiles[:5])

['CC(O)(P(=O)(O)O)P(=O)(O)O' 'CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C'
 'OC[C@H](O)[C@@H](O)[C@H](O)CO'
 'CCCCCCCC(=O)[O-].CCCCCCCC(=O)[O-].[Zn+2]' 'CC(C)COC(=O)C(C)C']


In [15]:
tokenizer = dc.feat.smiles_tokenizer.BasicSmilesTokenizer()

train_tok = list(map(tokenizer.tokenize, train_smiles))
valid_tok = list(map(tokenizer.tokenize, valid_smiles))
test_tok = list(map(tokenizer.tokenize, test_smiles))
print(train_tok[0])
len(train_tok)

['C', 'C', '(', 'O', ')', '(', 'P', '(', '=', 'O', ')', '(', 'O', ')', 'O', ')', 'P', '(', '=', 'O', ')', '(', 'O', ')', 'O']


6258

In [16]:
flatten = lambda l: [item for items in l for item in items]

all_toks = flatten(train_tok) + flatten(valid_tok) + flatten(test_tok)
vocab = sorted(set(all_toks + [""]))
print(vocab[:12], "...", vocab[-12:])
len(vocab)

['', '#', '(', ')', '-', '.', '/', '1', '2', '3', '4', '5'] ... ['[n+]', '[n-]', '[nH+]', '[nH]', '[o+]', '[s+]', '[se]', '\\', 'c', 'n', 'o', 's']


127

In [17]:
str2int = {s:i for i, s in enumerate(vocab)}
int2str = {i:s for i, s in enumerate(vocab)}
print(f"str2int: {dict(list(str2int.items())[:5])} ...")
print(f"int2str: {dict(list(int2str.items())[:5])} ...")

str2int: {'': 0, '#': 1, '(': 2, ')': 3, '-': 4} ...
int2str: {0: '', 1: '#', 2: '(', 3: ')', 4: '-'} ...


In [18]:
encode = lambda s: [str2int[tok] for tok in s]
decode = lambda i: [int2str[num] for num in i]
print(train_smiles[0])
print(encode(train_tok[0]))
print("".join(decode(encode(train_tok[0]))))

CC(O)(P(=O)(O)O)P(=O)(O)O
[19, 19, 2, 24, 3, 2, 25, 2, 16, 24, 3, 2, 24, 3, 24, 3, 25, 2, 16, 24, 3, 2, 24, 3, 24]
CC(O)(P(=O)(O)O)P(=O)(O)O


In [19]:
train_num = list(map(encode, train_tok))
valid_num = list(map(encode, valid_tok))
test_num = list(map(encode, test_tok))
print(train_num[0])

[19, 19, 2, 24, 3, 2, 25, 2, 16, 24, 3, 2, 24, 3, 24, 3, 25, 2, 16, 24, 3, 2, 24, 3, 24]


In [20]:
max_len = max(map(len, train_num + valid_num + test_num))
max_len

240

In [21]:
zero_pad = lambda x: x + [0] * (max_len - len(x))

train_numpad = np.array(list(map(zero_pad, train_num)))
valid_numpad = np.array(list(map(zero_pad, valid_num)))
test_numpad = np.array(list(map(zero_pad, test_num)))
train_numpad

array([[19, 19,  2, ...,  0,  0,  0],
       [19, 19,  2, ...,  0,  0,  0],
       [24, 19, 41, ...,  0,  0,  0],
       ...,
       [19, 19,  2, ...,  0,  0,  0],
       [19, 19,  2, ...,  0,  0,  0],
       [23,  1, 19, ...,  0,  0,  0]])

In [22]:
idx = np.random.randint(0, train_numpad.shape[0], 1).item()
print(train_smiles[idx])
print("".join(decode(train_numpad[idx])))

Cc1ccc(OC(=O)CC(C)C)cc1
Cc1ccc(OC(=O)CC(C)C)cc1


In [25]:
def tokenize_smiles(x, vocab, max_len):
    tokenizer = dc.feat.smiles_tokenizer.BasicSmilesTokenizer()
    str2int = {s:i for i, s in enumerate(vocab)}
    encode = lambda s: [str2int[tok] for tok in s]
    zero_pad = lambda x: x + [0] * (max_len - len(x))
    x = tokenizer.tokenize(x)
    x = encode(x)
    x = zero_pad(x)
    return np.array(x)

class SmilesFeaturizer(dc.feat.Featurizer):
    def __init__(self, feat_func, vocab, max_len):
        self.feat_func = feat_func
        self.vocab = vocab
        self.max_len = max_len

    def _featurize(self, x):
        return self.feat_func(x, self.vocab, self.max_len)

smiles_featurizer = SmilesFeaturizer(tokenize_smiles, vocab, max_len)

In [27]:
import numpy as np
import deepchem as dc

# Re-defining tokenize_smiles and SmilesFeaturizer within this cell
# to ensure the fix is applied directly where the error is processed.
def tokenize_smiles(x, vocab, max_len):
    tokenizer = dc.feat.smiles_tokenizer.BasicSmilesTokenizer()
    str2int = {s:i for i, s in enumerate(vocab)}
    encode = lambda s: [str2int[tok] for tok in s]
    zero_pad = lambda x_list: x_list + [0] * (max_len - len(x_list))
    try:
        tokens = tokenizer.tokenize(x)
        encoded_tokens = encode(tokens)
        padded_encoded_tokens = zero_pad(encoded_tokens)
    except (KeyError, Exception) as e:
        # For debugging: print(f"Error tokenizing SMILES '{x}': {e}")
        # Return a zero-padded array of the correct max_len if an error occurs
        padded_encoded_tokens = [0] * max_len
    return np.array(padded_encoded_tokens)

class SmilesFeaturizer(dc.feat.Featurizer):
    def __init__(self, feat_func, vocab, max_len):
        self.feat_func = feat_func
        self.vocab = vocab
        self.max_len = max_len

    def _featurize(self, x):
        # Ensure that vocab and max_len are passed correctly from the featurizer instance
        return self.feat_func(x, self.vocab, self.max_len)

# Create a new instance of the fixed featurizer
# The vocab and max_len variables are available in the kernel state from previous cells.
smiles_featurizer_fixed = SmilesFeaturizer(tokenize_smiles, vocab, max_len)

# Uninstall the current numpy version to ensure a clean slate
!pip uninstall -y numpy

# Install a compatible numpy version (e.g., numpy==1.23.0 to satisfy deepchem<2 requirement)
!pip install numpy==1.23.0

# Reinstall deepchem to ensure it uses the newly installed numpy version and avoids binary incompatibility
!pip install -qq --pre deepchem --upgrade --force-reinstall

tasks, datasets, transformers = dc.molnet.load_tox21(featurizer=smiles_featurizer_fixed)
print(datasets[0].X)

[05:04:07] WARNING: not removing hydrogen atom without neighbors
[05:04:08] Explicit valence for atom # 8 Al, 6, is greater than permitted
[05:04:08] Explicit valence for atom # 3 Al, 6, is greater than permitted
[05:04:08] Explicit valence for atom # 4 Al, 6, is greater than permitted
[05:04:08] Explicit valence for atom # 4 Al, 6, is greater than permitted
[05:04:09] Explicit valence for atom # 9 Al, 6, is greater than permitted
[05:04:09] Explicit valence for atom # 5 Al, 6, is greater than permitted
[05:04:09] Explicit valence for atom # 16 Al, 6, is greater than permitted
[05:04:10] Explicit valence for atom # 20 Al, 6, is greater than permitted


[[ 19  19   2 ...   0   0   0]
 [ 19  19   2 ...   0   0   0]
 [ 24  19  41 ...   0   0   0]
 ...
 [ 19  19  19 ...   0   0   0]
 [ 19  39   2 ...   0   0   0]
 [ 19 123   7 ...   0   0   0]]


Tokenization -> Vocabulary 생성 -> Numericalization -> Zero-Padding -> Embedding

# 💡 Insight & Reflection

Smiles / #D conformer을 이용해 물성 예측 다각화

쿨롱 Matrix(Eig)를 이용해 정전기적 상호작용 수치화 및 안정성 극대화

NLP를 도입하여 SMILES를 토큰화시켜 Transformer에 이식할 수 있게됨